In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import json
import re
from requests import get
import collections
import pandas as pd

In [3]:
rank_df = pd.read_csv("fifa_ranking.csv")

In [4]:
rank_groups = rank_df.groupby('rank_date')

In [5]:
wc2006 = rank_groups.get_group('5/17/06')
wc2010 = rank_groups.get_group('5/26/10')
wc2014 = rank_groups.get_group('5/8/14')
wc2018 = rank_groups.get_group('5/17/18')

In [6]:
wc2006

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
27758,1,Brazil,BRA,0.0,830,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,5/17/06
27759,2,Czech Republic,CZE,0.0,779,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,5/17/06
27760,3,Netherlands,NED,0.0,774,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,5/17/06
27761,4,Mexico,MEX,0.0,758,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,5/17/06
27762,5,USA,USA,0.0,760,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,5/17/06
27763,5,Spain,ESP,0.0,759,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,5/17/06
27764,7,Portugal,POR,0.0,753,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,5/17/06
27765,8,France,FRA,0.0,754,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,5/17/06
27766,9,Argentina,ARG,0.0,753,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,5/17/06
27767,10,England,ENG,0.0,744,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,5/17/06


In [7]:
tourney_df = pd.read_csv("WorldCupMatches.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa5 in position 7: invalid start byte